<a href="https://colab.research.google.com/github/HarshV-910/movie-recommend-app/blob/main/movie_recommendation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#basix of recommender system:
we can attract people/customer to our site according to their choise of interess useing this system.

##types of recommender system:
1. content based
 - here we are creating tags based on content type
 - ex. music related video on youtube, action movies in netflix
2. collaborative filtering based
- here we are creating group of users based on their intress
- ex. A and B watch movie X1 and like it. and A also liked movie X2, then we will recommend this movie to B,too.
3. hybrid
 - it is mixed of both, big companies like YouTube use this.

#**content based recommender system:**
##project flow:
    data -> preprocessing -> model -> website -> deploy

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

##data

In [2]:
movie = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')
print(movie.shape)
print(credit.shape)
movie.head(2)

(4803, 20)
(4803, 4)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [3]:
credit.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


##preprocessing

###merging both data based on title:

In [4]:
df = movie.merge(credit, on='title')
df.shape

(4809, 23)

###remove unnecessary columns which is not related to content of movie:
necessary columns like:genres,movie_id,keywords,title,overview,cast,crew

In [5]:
df = df[['movie_id','title','genres','keywords','overview','cast','crew']]
df.head(2)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


###creating new data in this format: [ id | title | tags ]
- where tags will creating using combination of genres,keywords,overview,cast,crew features

- cast : taking top 3 persons
- crew : taking only director

####finding and removing null values:

In [6]:
df.isnull().sum()

movie_id    0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [7]:
df.dropna(subset=['overview'],inplace=True)
df.isnull().sum()

movie_id    0
title       0
genres      0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

####removing duplicate data if available:

In [8]:
df.duplicated().sum()

np.int64(0)

####setting up genres & keywords columns in right format:

In [9]:
df.iloc[0].genres # it is string so giving error to next step

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [10]:
# extracting only name
import ast

def convert(obj):
  l = []
  # getting i as dictionary
  # for i in obj:  -> giving error: string indices must be integers, not 'str'
  for i in ast.literal_eval(obj): # now it become list form string
    l.append(i['name'])
  return l

df['genres'] = df['genres'].apply(convert)
df['keywords'] = df['keywords'].apply(convert)

In [11]:
df['genres'].sample(1)

1469    [Drama, Romance]
Name: genres, dtype: object

In [12]:
df['keywords'].sample(1)

2282    [new york, alcohol, sex, detective, newspaper,...
Name: keywords, dtype: object

####taking top 3 cast characters:

In [13]:
df.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [14]:
def convert3(obj):
  l = []
  counter = 0
  for i in ast.literal_eval(obj):
    l.append(i['name'])

    counter += 1
    if counter == 3:
      break

  return l

In [15]:
df['cast'] = df['cast'].apply(convert3)

In [16]:
df['cast'].head(1)

0    [Sam Worthington, Zoe Saldana, Sigourney Weaver]
Name: cast, dtype: object

####taking only director name from crew:

In [17]:
df.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [18]:
def fetch_director(obj):
  l = []
  counter = 0
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      l.append(i['name'])
      break

  return l

In [19]:
df['crew'] = df['crew'].apply(fetch_director)

In [20]:
df['crew'].head(1)

0    [James Cameron]
Name: crew, dtype: object

####converting overview into list:

In [21]:
df.iloc[0].overview

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [22]:
df['overview'] = df['overview'].apply(lambda x:x.split())
df['overview'].head(1).to_list()

[['In',
  'the',
  '22nd',
  'century,',
  'a',
  'paraplegic',
  'Marine',
  'is',
  'dispatched',
  'to',
  'the',
  'moon',
  'Pandora',
  'on',
  'a',
  'unique',
  'mission,',
  'but',
  'becomes',
  'torn',
  'between',
  'following',
  'orders',
  'and',
  'protecting',
  'an',
  'alien',
  'civilization.']]

####remove spaces between names:
if we have 2 names with different surname that can create problem.<br>
**ex:**<br> name1= harsh vekariya --> ['harsh' , 'vekariya']<br>
name2 = harsh gopani --> ['harsh' , 'gopani']<br>
here both list have harsh that can create relation and this is problem for us.

**solution:**

'Harsh Vekariya' --> 'HarshVekariya' --> ['HarshVekariya']

In [23]:
df['genres'] = df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
df['keywords'] = df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
df['cast'] = df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
df['crew'] = df['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [24]:
df.head(1)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


####Creating tags:

In [25]:
# concatination
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew']
final = df[['movie_id','title','tags']]

In [26]:
final.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [27]:
# join all elements of list with " "
final['tags'] = final['tags'].apply(lambda x:" ".join(x))

/tmp/ipykernel_9788/1207268534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags'] = final['tags'].apply(lambda x:" ".join(x))


In [28]:
final['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [29]:
# convert whole tags data to lowercase
final['tags'] = final['tags'].apply(lambda x:x.lower())

/tmp/ipykernel_9788/2656590810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags'] = final['tags'].apply(lambda x:x.lower())


##model

###first we have to find similarity between tags for all movies:

for this we should use vectorization. if we create vectors for each movie and to according to person selected movie, we have to recommand movies with nearest vectors.

different vectorization techniques for text to vector:
1. bag of words (we will use)
2. tfidf
3. word2vec

###bag of words: first concatinate all tags(tag1 + tag2 ...), and now it will vary very large text data, now find most 'n'(ex:5000) frequently used words.

now for 5000 words create table with 5000 columns like w1,w2,w3...w5000. and now according to tags of perticular movie fill this table with frequency of this selected words. at the end according to table we can select 'k' nearest vectors for recommendation.

###in vectorization first we have to neglet stop words from our tag data. like (is, are, if, and, too, from, to, in...)

###**for vectorization we have sklearn library called CountVectorizer(stop_words='english')**

###also we have to words in vectors like ['loved' , 'love' , 'loving'], so all have same meaning so that we have to choose only 1 among of them. for that we have to apply **stemming** in to tags it will convert this into same like ['love' , 'love' , 'love'].

In [30]:
# stemming

ps = PorterStemmer()

def stem(text):
  y = []
  for i in text.split(): # string to list of words
    y.append(ps.stem(i))

  return " ".join(y) # list to string

final['tags'] = final['tags'].apply(stem)

/tmp/ipykernel_9788/482919449.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags'] = final['tags'].apply(stem)


In [31]:
# vectorization

cv = CountVectorizer( stop_words='english',max_features=5000)
vectors = cv.fit_transform(final['tags']).toarray()

In [32]:
# we have 4860 movies and 5000 columns of words in vectors
vectors.shape

(4806, 5000)

In [33]:
cv.get_feature_names_out().tolist()[100:120]

['adrienbrodi',
 'adult',
 'adultanim',
 'adulteri',
 'adulthood',
 'advanc',
 'adventur',
 'adventure',
 'adventures',
 'advertis',
 'advic',
 'advis',
 'affair',
 'affect',
 'afghanistan',
 'africa',
 'african',
 'africanamerican',
 'aftercreditssting',
 'afterlif']

###now we find cosine distance(angle between 2 vectors) to find nearest 5 vectors:
- for this we have function in sklearn called cosine_similarity(vectors)

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors) # range is 0 to 1, where 0->no similarity(high distance), 1->high similarity(0 distance)

In [35]:
similarity.shape

(4806, 4806)

In [36]:
similarity[0] # similarities from first movie to all movies

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ], shape=(4806,))

###function to find first 5 similar movies for recommendation:

In [37]:
print(final[final['title'] == 'Avatar'])
print(final[final['title'] == 'Avatar'].index[0])
print()
print(similarity[0])
print(sorted(similarity[0],reverse=True))
print()
print(list(enumerate(similarity[0]))) # with index
print()
print(sorted(list(enumerate(similarity[0])), reverse=True)) # reverse sorting with index
print()
print(sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])) # reversing sorting with similarity
print()
print(sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]) # first 5


   movie_id   title                                               tags
0     19995  Avatar  in the 22nd century, a parapleg marin is dispa...
0

[1.         0.08346223 0.0860309  ... 0.04499213 0.         0.        ]
[np.float64(1.0000000000000002), np.float64(0.28676966733820225), np.float64(0.26901379342448517), np.float64(0.2605130246476754), np.float64(0.255608593705383), np.float64(0.25038669783359574), np.float64(0.24511108480187255), np.float64(0.24455799402225922), np.float64(0.2367785320221084), np.float64(0.23179316248638276), np.float64(0.23174488732966075), np.float64(0.2278389747471728), np.float64(0.2252817784447915), np.float64(0.21853668936906193), np.float64(0.21239769762143662), np.float64(0.2108663315950723), np.float64(0.2105263157894737), np.float64(0.20443988269091456), np.float64(0.20437977982832192), np.float64(0.20395079136182276), np.float64(0.2029530274475215), np.float64(0.2029530274475215), np.float64(0.20277677641345318), np.float64(0.2024645717996314), np

In [ ]:
def recommend(movie): # movie is title of given movie
  movie_index = final[final['title'] == movie].index[0] # finding index of given movie to access similarity list of its
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

  # fetch title of movies from movies_list
  for i in movies_list:
    print(final.iloc[i[0]].title)
    print()



In [39]:
recommend('Avatar')

Aliens vs Predator: Requiem

Aliens

Falcon Rising

Independence Day

Titan A.E.



In [ ]:
# we can't send pandas Dataframe dataset using pickle so we should have to convert into dictionary first

import pickle
pickle.dump(final.to_dict(),open('movies_dict.pkl','wb'))

In [41]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [42]:
final.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
